<a id="1"></a>
# Twitter Scraping using Tweepy

In [ ]:
import tweepy
import pymongo
import time
import datetime

In [ ]:
mongo_client = pymongo.MongoClient()
mongo_db = mongo_client["twitter"]
mongo_collection = mongo_db["tweets"]

def insert_into_mongo(collection, docs):
    try:
        collection.insert_many(docs, ordered = False )
    except Exception as e:
        pass


def remove_dublicates_mongo(mongo_db, old_collection_name, new_collection_name):
    """
    remove duplicated docs in mongoDB
    """
    try:
        old_collection = mongo_db[old_collection_name]
        new_collection = mongo_db[new_collection_name]
        
        old_cursor = old_collection.find()
        
        docs = [doc for doc in old_cursor]
        insert_into_mongo(new_collection, docs)
        old_collection.drop()
        
    except Exception as e:
        print('Exception : %s' % str(e))

In [ ]:
TWITTER_CONSUMER_KEY = "nCfs9fJCXThG9PHVWinsVywHt"
TWITTER_CONSUMER_SECRET = "EvFpByStx6diPJ0UZplSkhBd53laQj5lheE0KynGojEtTtAy17"
TWITTER_ACCESS_TOKEN = "1380480442818789376-xXSTw2pNEzHUgJ486YFkxPeXXkWSGX"
TWITTER_ACCESS_TOKEN_SECRET = "R7sSTfgCV3O2ZZryWtRNQZHTc8blOJN60zqhM46FG6ak9"

def create_api():
    consumer_key = TWITTER_CONSUMER_KEY
    consumer_secret = TWITTER_CONSUMER_SECRET
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET

    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, 
        wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except Exception as e:
        print("Error creating API", exc_info=True)
        raise e
    print("API created")
    return api


In [ ]:
def get_search_results(api, query):
    """
    get search results from Twitter using API
    """
    output = []

    for tweet in api.search(q=query, tweet_mode='extended', lang="en", count=100):
        output.append(tweet._json)

    return output


def remove_duplicates(l):
    """
    using list comprehension to remove duplicates 
    """
    return [i for n, i in enumerate(l) if i not in l[n + 1:]]

In [ ]:
def main():
    api = create_api()
        
    while True:
        tweets_no = 0
        print("Scraping new tweets...")
        tweets = get_search_results(api, query="covid")  
        new_key = "_id"
        old_key = "id_str"
        for i in tweets:
            i[new_key] = i.pop(old_key)
        print(len(tweets), " tweets found!")
        
        insert_into_mongo(mongo_collection, tweets)
        tweets_no = mongo_collection.count()
        print("Number of total tweets in MongoDB:", tweets_no)
        unique_tweets_no = len(mongo_collection.distinct("_id"))
        print("Number of unique tweets in MongoDB:", unique_tweets_no)
        
                
        if tweets_no >= 1000000:
            break
            
        time.sleep(3)
        print("*"*100)

main()

In [ ]:
def convert_str_to_datetime(str_date):
    return datetime.datetime.strptime(str_date, '%a %b %d %H:%M:%S +0000 %Y')

api = create_api()

startDate = datetime.datetime(2020, 5, 1)
endDate =   datetime.datetime.now()

tweets = []
tmpTweets = get_search_results(api, query="covid")

for tweet in tmpTweets:
    if convert_str_to_datetime(tweet['created_at']) < endDate and convert_str_to_datetime(tweet['created_at']) > startDate:
        tweets.append(tweet)

while (convert_str_to_datetime(tmpTweets[-1]['created_at']) > startDate):
    tmpTweets = get_search_results(api, query="covid")
    
    for tweet in tmpTweets:
        if convert_str_to_datetime(tweet['created_at']) < endDate and convert_str_to_datetime(tweet['created_at']) > startDate:
            tweets.append(tweet)


In [ ]:
tweets[0]